In [1]:
print("Hello World!")

# I'm born to test maia engine as whole. nothing else. 

Hello World!


In [2]:
import pandas as pd

In [3]:
path_name = '/Users/amirejibiilia/Desktop/mAIa/FullData.xlsx'
data = pd.read_excel(path_name, engine='openpyxl')
data

/Users/amirejibiilia/anaconda3/lib/python3.10/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


,date,client,metrics,value
0,2024-09-30,შპს თი ჯი ლიზინგი 402086924,შემოსავალი მომსახურების მიწოდებიდან 18%,6834.32
1,2024-08-23,შპს ჯორჯიან უოთერ ენდ ფაუერი 203826002,შემოსავალი საქონლის მიწოდებიდან 18%,203.39
2,2024-07-31,სს ვისოლ პეტროლიუმ ჯორჯია / 202161098,შემოსავალი მომსახურების მიწოდებიდან 18%,21691.97
3,2024-02-29,სს ნიკორა ტრეიდი 206255808,შემოსავალი მომსახურების მიწოდებიდან 18%,2950.61
4,2024-06-30,სს თიბისი ლიზინგი 205016560,შემოსავალი მომსახურების მიწოდებიდან 18%,10389.45
...,...,...,...,...
366,2024-08-31,სს ნიკორა ტრეიდი 206255808,შემოსავალი მომსახურების მიწოდებიდან 18%,3011.48
367,2024-07-08,შპს მაიჯიპიეს 205216176,შემოსავალი საქონლის მიწოდებიდან 18%,1525.42
368,2024-10-31,სს საქართველოს სახელმწიფო ელექტროსისტემა 20499...,შემოსავალი მომსახურების მიწოდებიდან 18%,3666.25
369,2024-03-27,შპს ჯავახავტოგზა 223352930,შემოსავალი საქონლის მიწოდებიდან 18%,1016.95


In [8]:
def query_data(data: pd.DataFrame, where: dict = None, group_by: list = None, 
               aggregations: dict = None, order_by: list = None) -> pd.DataFrame:
    """Main function that ties together all the SQL-like operations."""
    # Convert 'date' column to datetime if not already
    if 'date' in data.columns and not pd.api.types.is_datetime64_any_dtype(data['date']):
        data['date'] = pd.to_datetime(data['date'], format='%d.%m.%y')
    
    # Apply operations step by step
    data = apply_where(data, where)
    data = group_and_aggregate(data, group_by, aggregations)
    data = apply_order_by(data, order_by)
    
    return data

def apply_where(data: pd.DataFrame, where: dict) -> pd.DataFrame:
    """Applies WHERE conditions to the DataFrame."""
    if where:
        for column, condition in where.items():
            for operator, value in condition.items():
                # Convert value to datetime if the column is 'date' (or any datetime column)
                if column == "date" and isinstance(value, str):  
                    value = pd.to_datetime(value, format='%Y-%m-%d', errors='coerce')  
                    
                if operator == '>=':
                    data = data[data[column] >= value]
                elif operator == '<':
                    data = data[data[column] < value]
                elif operator == '=':
                    data = data[data[column] == value]
                elif operator == '!=':
                    data = data[data[column] != value]
                elif operator == '<=':
                    data = data[data[column] <= value]
                elif operator == '>':
                    data = data[data[column] > value]
    return data

def group_and_aggregate(data: pd.DataFrame, group_by: list, aggregations: dict) -> pd.DataFrame:
    """Groups the DataFrame by specified columns and applies aggregations, handling time periods."""
    if not group_by:
        # If no grouping is needed, simply apply the aggregation over the whole dataset
        data = pd.DataFrame(data.agg(aggregations)).transpose()
    else:
        # Process special time period groupings
        processed_group_by = []
        data_copy = data.copy()
        
        for group_col in group_by:
            # Handle special time period groupings
            if group_col == 'quarter':
                data_copy['quarter'] = data_copy['date'].dt.to_period('Q').astype(str)
                processed_group_by.append('quarter')
            elif group_col == 'month':
                data_copy['month'] = data_copy['date'].dt.to_period('M').astype(str)
                processed_group_by.append('month')
            elif group_col == 'year_only':
                data_copy['year_only'] = data_copy['date'].dt.year
                processed_group_by.append('year_only')
            elif group_col == 'week':
                data_copy['week'] = data_copy['date'].dt.isocalendar().week
                processed_group_by.append('week')
            else:
                # Regular column (metrics, client, or others)
                processed_group_by.append(group_col)
        
        # Perform grouping with processed columns
        data = data_copy.groupby(processed_group_by, as_index=False).agg(aggregations)

    # Rename columns to match SQL-style SELECT aliases
    new_columns = []
    
    for col in data.columns:
        if isinstance(col, tuple):  # Multi-index columns from aggregation
            if isinstance(col[1], list):
                # Handle case where aggregation is a list like ["sum"]
                new_columns.append(f"{col[0]}_{col[1][0]}")
            else:
                # Handle case where aggregation is a string like "sum"
                new_columns.append(f"{col[0]}_{col[1]}")
        else:
            new_columns.append(col)  # Keep original column name

    data.columns = new_columns  # Assign new column names
    return data

def apply_order_by(data: pd.DataFrame, order_by: list) -> pd.DataFrame:
    """Applies ORDER BY sorting to the DataFrame."""
    if order_by:
        for col, ascending in order_by:
            if isinstance(data, pd.DataFrame):  # Check if it's a DataFrame
                data = data.sort_values(by=col, ascending=ascending)
    return data

def execute_query(query_json: dict) -> pd.DataFrame:
    """
    Executes the query by receiving the general JSON-like object with all query parameters.
    The query_json structure should include 'data', 'where', 'group_by', 'aggregations', and 'order_by'.
    """
    # Extract parameters from the provided JSON object
    data = query_json.get("data")
    where = query_json.get("where")
    group_by = query_json.get("group_by")
    aggregations = query_json.get("aggregations")
    order_by = query_json.get("order_by")
    
    # Call query_data with the parameters extracted from the JSON-like object
    result = query_data(data, 
                        where=where, 
                        group_by=group_by, 
                        aggregations=aggregations, 
                        order_by=order_by)
    
    return result

In [9]:
execute_query({ "data": data, 
                "where": { "date": { ">=": "2024-01-01", "<=": "2024-03-31" } }, 
               "group_by": ["metrics"], "aggregations": { "value": ["mean"] }, 
               "order_by": [["value_mean", False]] })



,metrics_,value_mean
2,შემოსავალი ტაბლოები/სენსორები,87457.625000
1,შემოსავალი საქონლის მიწოდებიდან 18%,6406.921053
0,შემოსავალი მომსახურების მიწოდებიდან 18%,5387.484314
3,შემოსავალი ტექნიკური მომსახურებიდან,322.882000


In [10]:
execute_query({ "data": data, 
               "where": { "date": { ">=": "2024-01-01", "<=": "2024-03-31" } }, 
               "group_by": ["year_only"], 
               "aggregations": { "value": ["sum"] } })

,year_only_,value_sum
0,2024,573022.86


In [11]:
execute_query({ "data": data, 
               "where": { "client": { "=": "სს ენერგო პრო გორგია 205169066" } },  
               "group_by": ["year_only"], 
               "aggregations": { "value": ["sum"] } })

,year_only_,value_sum
